In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf

Init Plugin
Init Graph Optimizer
Init Kernel


## Importing data

In [4]:
df = pd.read_csv("data/fer2013.csv")

In [6]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [8]:
df.Usage.unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

In [15]:
df.Usage.value_counts()

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64

In [21]:
# Combining PublicTest and PrivateTest to the whole testing set
df_training = df[df["Usage"] == "Training"]
df_testing = df[df["Usage"] != "Testing"]

In [26]:
# Save training set and testing set as csv file for the purpose of futher use
# Uncomment commands below if you need to save these files
#df_training.to_csv("data/train.csv", index = False)
#df_testing.to_csv("data/test.csv", index = False)

In [60]:
# create inputs and labels for the training set
def get_inputs_labels(file_path):
    df = pd.read_csv(file_path)
    inputs = [np.array(df["pixels"][i].split(" ")) for i in range(df.shape[0])]
    labels = [df["emotion"][i] for i in range(df.shape[0])]
    assert len(inputs) == len(labels), "inputs and labels should have the same length."
    return np.array(inputs), np.array(labels)

In [63]:
def training_testing_prep(training_file_path, testing_file_path):
    image_size = 48
    training_inputs, training_labels = get_inputs_labels(training_file_path)
    testing_inputs, testing_labels = get_inputs_labels(testing_file_path)
    training_inputs = training_inputs.reshape(training_inputs.shape[0], image_size, image_size, 1)
    testing_inputs = testing_inputs.reshape(testing_inputs.shape[0], image_size, image_size, 1)
    training_labels = tf.one_hot(training_labels, 7, dtype=tf.float32)
    testing_labels = tf.one_hot(testing_labels, 7, dtype=tf.float32)
    
    return training_inputs, training_labels, testing_inputs, testing_labels

In [57]:
len(labels)

28709

In [59]:
np.array(inputs).shape

(28709, 2304)